In [61]:
import getNewReading as gnr
import kinneretDrawGraph as kdg
import pandas as pd
import math

In [62]:
df = gnr.importReadings()
df

,level
date,
2020-12-27,-209.900
2020-12-26,-209.895
2020-12-25,-209.890
2020-12-24,-209.885
2020-12-23,-209.890
...,...
1966-11-01,-210.390
1966-10-15,-210.320
1966-10-01,-210.270


In [80]:
df['real'] = True

upsampled = df.resample('1D')
interpolated = upsampled.interpolate(method='polynomial', order=5)
interpolated.sort_values(by='date', inplace=True, ascending=False)

In [81]:
interpolated.fillna(value=False, inplace=True)
interpolated.tail(50)

,level,real
date,,
1966-10-20,-210.340328,False
1966-10-19,-210.336042,False
1966-10-18,-210.331847,False
1966-10-17,-210.327766,False
1966-10-16,-210.323813,False
1966-10-15,-210.320000,True
1966-10-14,-210.316328,False
1966-10-13,-210.312791,False
1966-10-12,-210.309375,False


In [ ]:
dataFile = 'data/levels-pd-calc2.csv'
colHead = ['level','1day','7day','1month']
df.round({'1day': 3,'7day': 3, '1month': 3}).to_csv(dataFile, index_label='date', columns=colHead)

In [ ]:
dateOff = pd.DateOffset(days=-1)
ind = df.index[5]
# df['1dayT'] = [gnr.naCheckDelta(df,x,y,dateOff1d) for x,y in zip(df.index,df['1day'])]
# df['gap'] = [x for x in df.]
timeAgo = dateOff + ind
filt = df.index.get_loc(timeAgo, method='nearest')
oldLevel = df.iloc[filt]

print(oldLevel)